In [ ]:
def pca(X, pc):
    # demean X
    
    # get covariance of x
    
def whitening(X, eps=10e-6):
    # X = [m,n]
    # returns transformation for mapping
    covX = np.dot(X,X.T) / len(X)
    
    w, v = linalg.eig(covX)
    w = np.real(w)
    idx = w > eps
    w = np.sqrt(w[idx])
    v = np.real(v[:,idx])

    ## sort by decreasing order of eigen vals
    #     idx = w.argsort()[::-1]
    #     w = w[idx]
    #     v = v[:,idx]
    w_inv = 1 / w 

    transform = np.dot(np.dot(v, np.diag(w_inv)), v.T)
    
    return transform